In [5]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from dotenv import load_dotenv
import os

load_dotenv("./.env")


True

In [6]:
import os
from typing import List, Any

def read_file(file_path: str) -> List[Any]:
    """
    Read a file and return its content using appropriate LangChain document loader.
    
    Args:
        file_path (str): Path to the file to be read
        
    Returns:
        List[Any]: List of documents loaded from the file
        
    Raises:
        FileNotFoundError: If the file doesn't exist
        ValueError: If the file type is not supported
    """
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    # Get file extension
    file_extension = os.path.splitext(file_path)[1].lower()
    
    # Choose appropriate loader based on file extension
    if file_extension == '.pdf':
        loader = PyPDFLoader(file_path)
    elif file_extension == '.txt':
        loader = TextLoader(file_path)
    elif file_extension in ['.docx', '.doc']:
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")
    
    # Load and return documents
    try:
        documents = loader.load()
        return documents
    except Exception as e:
        raise Exception(f"Error loading file: {str(e)}")

# Example usage function
def read_and_display_content(file_path: str) -> None:
    """
    Read a file and display its content.
    
    Args:
        file_path (str): Path to the file to be read
    """
    try:
        documents = read_file(file_path)
        print(f"Successfully loaded {len(documents)} document(s) from: {file_path}")
        
        return documents
                
    except Exception as e:
        print(f"Error: {str(e)}")

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=500
)


In [10]:
from langchain_mistralai.chat_models import ChatMistralAI
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
chat_model = ChatMistralAI(
    model_name="mistral-small-latest",
    temperature=0.7,
    api_key=MISTRAL_API_KEY
)


In [18]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant that helps to extract report from logistics data."),
    HumanMessagePromptTemplate.from_template("""
                                            Please extract the key informations from the following logistics report. In defined output format.
                                            If any of the key information is missing, fill in with "N/A"
                                            -Key Informations:{logistics_report}
                                            -output_format:
                                             
                                            `json
                                            {{
                                             delivery_time: list[str]
                                             delays: list[str]
                                             route_issues: list[str]
                                            }}
                                             
                                            {error_context}
                                            """)
])

print("Prompt template created successfully!")
print(f"Template messages: {len(prompt_template.messages)}")

Prompt template created successfully!
Template messages: 2


In [19]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class SummaryResponse(BaseModel):
    delivery_time: list[str] = Field(default_factory=list)
    delays: list[str] = Field(default_factory=list)
    route_issues: list[str] = Field(default_factory=list)

output_parser = PydanticOutputParser(pydantic_object=SummaryResponse)

In [25]:

lcel = prompt_template | chat_model | output_parser

def chain_with_retry(logistics_report : str, attempts: int = 3):

    error_context = ""

    for attempt in range(attempts):

        try:
            input_data = {
                "logistics_report": logistics_report,
                "error_context": error_context
            }
            result = lcel.invoke(input_data)
            return result
        except Exception as e:
            error_context = "Previous attempts failed with error: " + str(e)
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == attempts - 1:
                raise Exception("All attempts to process the logistics report have failed."+str(e))

    return None

In [26]:
result = chain_with_retry(logistics_report="""Truck 204 departed Bangalore at 10:30 AM.
Delay of 3 hours due to highway construction near Hosur.
Consignment #556 delivered in Chennai at 4:00 PM.""")